# VTK Algorithm Source for Lazy Evaluation

pyvista-xarray provides a `PyVistaXarraySource` VTK algorithm that lazily evaluates xarray DataArrays on demand. This is ideal for:

- **Large datasets** that don't fit in memory
- **Dask-backed** arrays that should only be computed when needed
- **Time series** data where you want to step through timesteps interactively
- **Level-of-detail** rendering where you downsample for fast interaction

This notebook demonstrates these capabilities using xarray tutorial datasets.

In [ ]:
import pyvista as pv
import xarray as xr

import pvxarray  # noqa: F401

## Time Stepping with Air Temperature

The `.pyvista.algorithm()` method creates a VTK source that can be added to a plotter. Specify a `time` dimension to enable time stepping:

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
da = ds.air

# Create an algorithm source with time stepping
algorithm = da.pyvista.algorithm(x="lon", y="lat", time="time")

print(f"Time dimension has {algorithm.max_time_index + 1} steps")
print(f"Current time index: {algorithm.time_index}")

In [ ]:
# The algorithm can be added directly to a plotter
pl = pv.Plotter()
pl.add_mesh(algorithm)
pl.show(cpos="xy")

## 3D Basin Mask Visualization

The `basin_mask` tutorial dataset has X, Y, and Z coordinates, making it a true 3D dataset. Here we select a region of interest and visualize it as a 3D volume with vertical exaggeration:

In [ ]:
ds = xr.tutorial.load_dataset("basin_mask")
da = ds.basin

# Define a region of interest (bottom, left, top, right)
box = (-96.70, 90 - 64.98, -7.40, 90 + 23.18)
min_lat, min_lon, max_lat, max_lon = box

# Slice the data to the region of interest
roi = da.loc[{"Y": slice(min_lat, max_lat), "X": slice(min_lon, max_lon)}]
roi

In [ ]:
mesh = roi.pyvista.mesh(x="X", y="Y", z="Z")

pl = pv.Plotter()
pl.add_mesh(mesh)
pl.set_scale(zscale=0.001)  # Exaggerate the vertical axis
pl.show()